# Connected Components

The purpose of this assignment is to familiarize yourself with the handling of graph data structures. You will implement depth-first search for identifying the connected components of an undirected graph, implementing procedure Search as a subroutine along the way.

You will use the [NetworkX](https://networkx.github.io/) Python package to represent and manipulate graphs. You should first familiarize yourself with its functionality by going through the brief [tutorial](http://networkx.github.io/documentation/networkx-1.9.1/tutorial/index.html). For this homework, you may only use the basic undirected graph methods listed [here](http://networkx.github.io/documentation/networkx-1.9.1/reference/classes.graph.html).

As a use case, we will work with a dataset recording the interactions between characters in Homer's *Iliad*.

In [61]:
import networkx as nx
from urllib.request import urlopen
import re
import itertools
import numpy as np
import pandas as pd
from queue import *

In [62]:
hm = urlopen('http://people.sc.fsu.edu/~jburkardt/datasets/sgb/homer.dat')
homer = hm.read().decode('utf-8')

The format of the data is straightforward. After some comment lines (beginning with \*), the file lists a codename for each character (i.e., node of the graph), followed by a description. The file then lists the groups of characters that interact in each chapter, from which you will form the edges. For instance, the first line has the form:

```1:CH,AG,ME,GS;AP,CH;HE,AC;AC,AG,CA;HE,AT;AT,AC;AT,OG;NE,AG,AC;CS,OD```

This means that CH,AG,ME,GS interacted, so there are edges for all pairs of these nodes. Groups of characters that interacted are separated by semicolons. The lines start with chapter information of the form `1:` or `&:`, which can be ignored for this problem.

First implement a function to read in the nodes from the input file. You may implement any auxiliary functions as needed, and are encouraged to use small functions with specific purposes to keep your code readable. Any function you implement should be clearly commented.

In [63]:
def read_nodes(gfile):
    """Reads in the nodes of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top.
        
    Returns:
        A generator of the nodes in the graph, yielding a list of the form:
            ['CH', 'AG, 'ME', ...]
    """
    # TODO: implement function
    pattern = '\\n([A-Z0-9]{2})\s'
    regex = re.compile(pattern)
    nodes = regex.findall(gfile)
    return nodes
    pass

Next implement a function to read in the edges from the input file.

In [64]:
def read_edges(gfile):
    """Reads in the edges of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top 
            of the edges section.
            
    Returns:
        A generator of the edges in the graph, yielding a list of pairs of the form:
            [('CH', 'AG'), ('AG', 'ME'), ...]
    """
    # TODO: implement function
    pattern = '[:](.+?)[\n]'
    regex = re.compile(pattern)
    s1 = regex.findall(gfile)
    edgenodes =[]
    edges = []
    for i in range(len(s1)):
        for edgeset in s1[i].split(';'):
            s2 = (edgeset).split(',')
            edgenodes.append(s2)
    for eachset in edgenodes:
        edges = edges + list(itertools.combinations(eachset,2))
    return edges
    pass

The following code should now correctly create the graph.

In [65]:
G = nx.Graph()
G.add_nodes_from(read_nodes(homer))
G.add_edges_from(read_edges(homer))

Next implement procedure Search. The function takes in a graph and a root node, and returns a list of the nodes visited during the search. The nodes should appear in the order in which they were *first visited*. The neighbors of a node should be processed in *alphabetical order*, where numbers come before letters. This will ensure that the output of your function is uniquely defined, given any input node.

In [69]:
def bfs(graph, root):
    """Runs breadth-first search through a graph, starting at a given root. Neighboring
    nodes are processed in alphabetical order.
    
    Args:
        graph: the given graph, with nodes encoded as strings.
        root: the node from which to start the search.
        
    Returns:
        A list of nodes in the order in which they were first visited.
    """
    # TODO: implement function
    # using data structure queue in python
    discovered = dict.fromkeys(graph.nodes(),0)
    dist = dict.fromkeys(graph.nodes(),float('inf'))
    parent = dict.fromkeys(graph.nodes(),np.nan)
    q = Queue(maxsize=0)
    discovered[root] = 1
    dist[root] = 0
    parent[root] = np.nan
    q.put(root)
    visited=[]
    while q.empty()==False:
        u = q.get()
        visited.append(u)
        neigh = sorted(graph.neighbors(u))
        for v in neigh:
        #for (u,v) in graph.edges():
            if discovered[v] == 0:
                discovered[v] = 1
                dist[v] = dist[u] + 1
                parent[v] = u
                q.put(v)
                visited.append(v)
    return visited           
    pass

In [71]:
def bfs(graph, root):
    """Runs breadth-first search through a graph, starting at a given root. Neighboring
    nodes are processed in alphabetical order.
    
    Args:
        graph: the given graph, with nodes encoded as strings.
        root: the node from which to start the search.
        
    Returns:
        A list of nodes in the order in which they were first visited.
    """
    # TODO: implement function
    # without using data structure queue
    discovered = dict.fromkeys(graph.nodes(),0)
    dist = dict.fromkeys(graph.nodes(),float('inf'))
    parent = dict.fromkeys(graph.nodes(),np.nan)
    q = []
    discovered[root] = 1
    dist[root] = 0
    parent[root] = np.nan
    q.append(root)
    visited=[root]
    while len(q) >0:
        u = q.pop(0)
        neigh = sorted(graph.neighbors(u))
        for v in neigh:
        #for (u,v) in graph.edges():
            if discovered[v] == 0:
                discovered[v] = 1
                dist[v] = dist[u] + 1
                parent[v] = u
                q.append(v)
                visited.append(v)
    return visited           
    pass

We will check the correctness of your code by verifying that it correctly computes the DFS tree starting at Ulysses (node `OD`).

In [72]:
ulysses = bfs(G, 'OD')

Next implement DFS to find the connected components of the character graph. When choosing roots for your components, always pick the *smallest unvisited node* according to alphabetical ordering. Combined with your Search routine, this will ensure that the output is again uniquely defined.

In [73]:
def connected_components(graph):
    """Computes the connected components of the given graph.
    
    Args: 
        graph: the given graph, with nodes encoded as strings.
        
    Returns:
        The connected components of the graph. Components are listed in
        alphabetical order of their root nodes.
    """
    # TODO: implement function
    nodes = sorted(graph.nodes())
    SCC=[]
    explored = dict.fromkeys(nodes,0)
    def explore(x):
        explored[x]=1
    for u in nodes:
        if explored[u]==0:
            s = bfs(graph,u)
            [explore(i) for i in s]
            SCC.append(s)
    return SCC
    pass

We will check correctness of your code by verifying that your output list is identical to our solution.

In [74]:
character_interactions = connected_components(G)

In [75]:
for a in range(len(character_interactions)):
    print(len(character_interactions[a]),len(set(character_interactions[a])))

542 542
2 2
3 3
1 1
2 2
1 1
1 1
2 2
1 1
1 1
3 3
2 2


As a preliminary check, you should find that the following statements are all true.

In [76]:
component_sizes = [len(c) for c in character_interactions]
print("There are 12 connected components in the Iliad:", len(component_sizes) == 12)
print("The giant component has size 542:", max(component_sizes) == 542)
print("There are 5 isolated characters:", len([c for c in component_sizes if c == 1]) == 5)

There are 12 connected components in the Iliad: True
The giant component has size 542: True
There are 5 isolated characters: True


In [77]:
def is_bipartite(graph):
    # TODO: implement function
    # without using data structure queue
    discovered = dict.fromkeys(graph.nodes(),0)
    dist = dict.fromkeys(graph.nodes(),float('inf'))
    parent = dict.fromkeys(graph.nodes(),np.nan)
    color = dict.fromkeys(graph.nodes(),0)
    q = []
    nodes = sorted(graph.nodes())
    for u in nodes:
        if discovered[u]==0:
            discovered[u] = 1
            dist[u] = 0
            parent[u] = np.nan
            q.append(u)
            visited=[u]
        while len(q) >0:
            u = q.pop(0)
            color[u] = 'red'
            neigh = sorted(graph.neighbors(u))
            for v in neigh:
            #for (u,v) in graph.edges():
                if discovered[v] == 0:
                    discovered[v] = 1
                    dist[v] = dist[u] + 1
                    parent[v] = u
                    color[v] = "white"
                    q.append(v)
                    visited.append(v)
            for a,b in neigh:
                if (a,b) in graph.edges():
                    return "Graph is not bi-partite"
                    break
                else:
                    pass
    
    if sum(discovered.values())==len(nodes):
        print("Graph is bi-partite")

In [78]:
is_bipartite(G)

Graph is bi-partite
